## Package load

In [3]:
import numpy as np
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler


ModuleNotFoundError: No module named 'numpy'

## Model training

#### Read Training dataset

In [2]:
train_data=pd.read_csv('Train data.csv')
train_data['DateTime'] = pd.to_datetime(train_data['DateTime'])
train_data['Year'] = train_data['DateTime'].dt.year
train_data['Month'] = train_data['DateTime'].dt.month
train_data['Day'] = train_data['DateTime'].dt.day
train_data['Week'] = train_data['DateTime'].dt.dayofweek
train_data['Load_prev_day']=train_data['Load'].shift(24)
train_data=train_data[24:]

#### Process Training dataset

In [4]:
#Process the data
T_feature_train=np.array(train_data['Temperature']).reshape(-1,24)
H_feature_train=np.array(train_data['Humidity']).reshape(-1,24)
L_prev_feature_train=np.array(train_data['Load_prev_day']).reshape(-1,24)
W_feature_train=np.array(train_data['Week']).reshape(-1,24)[:,0:1]
M_feature_train=np.array(train_data['Month']).reshape(-1,24)[:,0:1]

# Traning dataset
X_train=np.concatenate((T_feature_train, H_feature_train,L_prev_feature_train,W_feature_train,M_feature_train), axis=1)
y_train=np.array(train_data['Load']).reshape(-1,24)

#Data Normalization
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train_nor = scaler_X.fit_transform(X_train)
y_train_nor = scaler_y.fit_transform(y_train)

In [9]:
pd.DataFrame(X_train_nor)

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,0.310361,0.311026,0.314822,0.318302,0.317943,0.316931,0.314032,0.311250,0.292229,0.274713,...,0.592837,0.637563,0.669816,0.653738,0.635141,0.630630,0.617355,0.617726,0.500000,0.0
1,0.288214,0.287282,0.289363,0.291229,0.293325,0.295168,0.292628,0.299668,0.283048,0.263606,...,0.703441,0.721429,0.727444,0.710343,0.689766,0.666259,0.671152,0.653159,0.666667,0.0
2,0.284420,0.287282,0.291750,0.292992,0.294374,0.299959,0.300855,0.301830,0.286159,0.272047,...,0.713319,0.728560,0.735344,0.704117,0.686678,0.687006,0.677810,0.028079,0.833333,0.0
3,0.332198,0.336886,0.343066,0.345214,0.347647,0.352253,0.349653,0.347927,0.335939,0.332099,...,0.055882,0.063475,0.058451,0.053445,0.050895,0.044563,0.036022,0.026653,1.000000,0.0
4,0.417841,0.415328,0.413955,0.417701,0.421987,0.427203,0.419775,0.410779,0.392852,0.380822,...,0.045240,0.053700,0.050597,0.047640,0.045561,0.037790,0.028933,0.646579,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,0.000000,0.000000,0.002148,0.006568,0.011058,0.017134,0.019407,0.030654,0.025118,0.015476,...,0.795829,0.831802,0.847228,0.804107,0.791977,0.768677,0.740054,0.790062,0.000000,1.0
726,0.069924,0.070371,0.072957,0.076572,0.080232,0.087942,0.094561,0.104625,0.097966,0.095594,...,0.889522,0.887147,0.907489,0.875325,0.854108,0.810860,0.784694,0.821729,0.166667,1.0
727,0.163389,0.162997,0.169544,0.172527,0.175398,0.180674,0.180177,0.181839,0.166717,0.156905,...,0.913473,0.924253,0.922819,0.905544,0.868402,0.829482,0.795936,0.836488,0.333333,1.0
728,0.198002,0.186271,0.179171,0.170124,0.165227,0.169143,0.166680,0.170798,0.167628,0.168160,...,0.872658,0.880960,0.904357,0.876835,0.862921,0.822958,0.793904,0.838962,0.500000,1.0


#### Model training

In [205]:
# Create a linear regression model
#74 24  24+24+6*24+2
linear_model = LinearRegression()

# Create a multioutput regressor using the linear regression model
multioutput_model = MultiOutputRegressor(linear_model)
multioutput_model.fit(X_train_nor, y_train_nor)

MultiOutputRegressor(estimator=LinearRegression())

In [206]:
# Evaluate the model accuracy at training dataset
y_train_pred = scaler_y.inverse_transform(multioutput_model.predict(X_train_nor))
print('ACC:',1-mean_absolute_percentage_error(y_train,y_train_pred))

ACC: 0.9627544482631327


## Model Evaluation

#### Read the test dataset

In [10]:
# Read test file
test_file_name=['February','April','June','August','October','December']
X_test=[]
for name in test_file_name:
    test_data=pd.read_csv('./Test/'+name+'.csv')
    test_data['DateTime'] = pd.to_datetime(test_data['DateTime'])
    test_data['Year'] = test_data['DateTime'].dt.year
    test_data['Month'] = test_data['DateTime'].dt.month
    test_data['Day'] = test_data['DateTime'].dt.day
    test_data['Week'] = test_data['DateTime'].dt.dayofweek
    vars()[name]=test_data[24:] # your can use this sentence to generate the variabels 'February','April',...


In [ ]:
test_data=pd.read_csv('./Test/October.csv')
test_data['DateTime'] = pd.to_datetime(test_data['DateTime'])
test_data['Year'] = test_data['DateTime'].dt.year
test_data['Month'] = test_data['DateTime'].dt.month
test_data['Day'] = test_data['DateTime'].dt.day
test_data['Week'] = test_data['DateTime'].dt.dayofweek
vars()[name]=test_data[24:]

#### Process the test dataset

In [ ]:
for name in test_file_name:
    test_data=vars()[name]
    T_feature_test=np.array(test_data[-24:]['Temperature'])
    H_feature_test=np.array(test_data[-24:]['Humidity'])
    L_prev_feature_test=np.array(test_data['Load'][-48:-24])
    W_feature_test=np.array(test_data[-24:]['Week'])[0:1]
    M_feature_test=np.array(test_data[-24:]['Month'])[0:1]
    X_test.append(np.concatenate((T_feature_test, H_feature_test,L_prev_feature_test,W_feature_test,M_feature_test)))

#### Output the Answer

In [267]:
# Inverse normalize
X_test_nor=scaler_X.transform(X_test)
y_pred_nor = multioutput_model.predict(X_test_nor)
y_pred_test = scaler_y.inverse_transform(y_pred_nor).reshape(1,-1)[0]

#Output the Answer
Answer=pd.read_csv('./Test/Answer.csv')
Answer['Load']=y_pred_test
Answer.to_csv('./Test/Answer.csv',index=False)